# Gabor layer experiment

> In this quick experiment we will be training an MNIST classifier using `GaussianLayer` layers.

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

## Library importing

In [ ]:
#| hide
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange, repeat

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

from flayers.center_surround import GaussianLayer

## Data loading

> We will be using MNIST for a simple and quick test.

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

X_train = repeat(X_train, "b h w ->  b h w c", c=1)/255.0
X_test = repeat(X_test, "b h w ->  b h w c", c=1)/255.0

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((60000, 28, 28, 1), (60000,), (10000, 28, 28, 1), (10000,))

## Definition of simple model

In [ ]:
filters = 4
sigma_i = [0.1, 0.2]*2
sigma_j = [0.2, 0.1]*2
freq = [10, 10]*2
rot_theta = [0, 0]*2
sigma_theta = [0, 0]*2

In [ ]:
model = tf.keras.Sequential([
    GaussianLayer(filters=filters, size=20, imean=0.5, jmean=0.5, sigma_i=sigma_i, sigma_j=sigma_j, freq=freq, 
                  rot_theta=rot_theta, sigma_theta=sigma_theta, fs=20, input_shape=(28,28,1)),
    layers.MaxPool2D(2),
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation="softmax")
])
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

2022-10-03 11:30:00.479576: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5435 MB memory:  -> device: 0, name: NVIDIA GeForce GTX TITAN Black, pci bus id: 0000:83:00.0, compute capability: 3.5
2022-10-03 11:30:02.969453: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-10-03 11:30:03.183661: I tensorflow/core/util/cuda_solvers.cc:180] Creating CudaSolver handles for stream 0x563cf94cc6e0


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gaussian_layer (GaussianLaye (None, 28, 28, 4)         1622      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 4)         0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 10)                50        
Total params: 1,672
Trainable params: 72
Non-trainable params: 1,600
_________________________________________________________________


We can show the initial Gabor filters:

In [ ]:
model.layers[0].show_filters()

In [ ]:
history = model.fit(X_train, Y_train, batch_size=128, epochs=1, validation_split=0.2)

2022-10-03 11:30:18.950268: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100
2022-10-03 11:30:19.480448: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


375/375 [==============================] - 17s 31ms/step - loss: 2.4841 - accuracy: 0.0947 - val_loss: 2.2841 - val_accuracy: 0.1020


We can visualize the gabor filters after the training process:

In [ ]:
model.layers[0].show_filters()

We can even check the atributes of the layer to inspect the change in the initial parameters:

In [ ]:
model.layers[0].rot_theta.numpy()*180/np.pi

array([-1.3243861 ,  0.27530777,  2.329611  , -1.5420123 ], dtype=float32)

In [ ]:
model.layers[0].sigma_theta.numpy()*180/np.pi

array([-12.016365 , -23.456697 ,  -6.973554 ,  -2.7918148], dtype=float32)